In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import gc
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/train.csv')
X_test =  pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/test.csv')
train.pop('id')

y = train['claim']
train.pop('claim')
X_test.pop('id')
X=train
X['sum_na'] = X.isna().sum(axis=1)
X_test['sum_na'] = X_test.isna().sum(axis=1)
del train
gc.collect()


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense,   Dropout,  Concatenate, Embedding,  Flatten, Add, Average
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler,  QuantileTransformer,  KBinsDiscretizer
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from sklearn import metrics

from sklearn.impute import SimpleImputer
#import tensorflow_decision_forests as tfdf

In [ ]:
def ClassModel(input_shape):

    

    X_input_bin = Input(input_shape)
    X_input = Input(input_shape)
    X = Embedding (input_dim=96, output_dim=18)(X_input_bin)
    #X = Dropout(0.4)(X)
    X = Flatten()(X)

    
    X = Dense(30, activation='swish')(X)
    X1 = Dense(30, kernel_initializer=tf.keras.initializers.GlorotNormal(), activation='swish')(X_input)
    X = Add()([X,X1])
    X = Dropout(0.5)(X)
    X = Dense(30, kernel_initializer=tf.keras.initializers.GlorotNormal(), activation='swish')(X)
#     X = BatchNormalization()(X)
    X = Dropout(0.5)(X)

    X = Dense(1, kernel_initializer=tf.keras.initializers.GlorotNormal(),activation='sigmoid', name='output2')(X)
    model = Model(inputs = [X_input_bin,X_input], outputs = X, name='ClassModel')

    return model


## 10 - fold section

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='median')
X = imp.fit_transform(X)
X_test = imp.transform(X_test)
qt = QuantileTransformer(n_quantiles=96, output_distribution='uniform')
X = qt.fit_transform(X)
X_test = qt.transform(X_test)
bin_cat = KBinsDiscretizer(n_bins=96, encode='ordinal',strategy='uniform')
X_bin = bin_cat.fit_transform(X)
X_test_bin = bin_cat.transform(X_test)

gc.collect()
def prediction (X_train_bin, X_train, y_train, X_test_bin, X_test):
    
    keras.backend.clear_session()

    kfold = StratifiedKFold(n_splits = 8, random_state=2021, shuffle=True)

    y_pred = np.zeros((493474,1))
    BATCH_SIZE=512
    SHUFFLE_BUFFER_SIZE = 1024

    for idx in kfold.split(X=X_train, y=y_train):
        train_idx, val_idx = idx[0], idx[1]
        xtrain = X_train[train_idx]
        xtrain_bin = X_train_bin[train_idx]
        ytrain = y_train[train_idx]
        xval = X_train[val_idx]
        xval_bin = X_train_bin[val_idx]
        yval = y_train[val_idx]



        
        checkpoint_filepath = '/kaggle/working/ckpt_cv'
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_aucroc',
    mode='max',
    save_best_only=True)
        
        
#         train_dataset = tf.data.Dataset.from_tensor_slices((np.float32(xtrain), np.float32(ytrain)))
#         val_dataset = tf.data.Dataset.from_tensor_slices((np.float32(xval), np.float32(yval)))
#         train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
#         val_dataset = val_dataset.batch(BATCH_SIZE)
        
        keras.backend.clear_session()
        # fit model for current fold
        classmodel = ClassModel(xtrain.shape[1:])
        classmodel.compile(loss='binary_crossentropy', optimizer = keras.optimizers.Adam(learning_rate=0.00012), metrics=[tf.keras.metrics.AUC(name='aucroc')])        
        classmodel.fit([xtrain_bin,xtrain],ytrain,batch_size=512,
        epochs = 15,
        validation_data=([xval_bin,xval],yval),
        callbacks=[model_checkpoint_callback])
        keras.backend.clear_session()
        classmodel.load_weights(checkpoint_filepath)
        #create predictions
        y_pred += classmodel.predict([X_test_bin,X_test])/kfold.n_splits
        del classmodel
        gc.collect()
    return y_pred

In [ ]:
# BATCH_SIZE=1024
# test_dataset = tf.data.Dataset.from_tensor_slices(np.float32(X_test))
# test_dataset = test_dataset.batch(BATCH_SIZE)
main_pred = prediction(X_bin,X, y, X_test_bin, X_test)

In [ ]:
main_pred[:10]

In [ ]:
sub = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv')
sub.iloc[:,1]=main_pred
sub=sub.set_index('id')
sub.to_csv('baseline_nn_cv.csv')